# Updating my Custom Items Datapack for 1.21.4

Welp, Minecraft 1.21.4 changed everything _again_ when it comes to custom model data, and now I have to update, not just the datapack, but the **resource pack** as well.

In [1]:
import json
from pathlib import Path

from IPython.display import Markdown, display

## Data Pack First

So the new syntax is:

```
components:{"minecraft:custom_model_data":{"floats":[CMD]}, ...}
```

The selectors can now also be strings, which will make everything more obvious and readible, but because I don't want to go around my whole world updating item NBT, I'm sticking with a straight port.

### Covfefes

In [2]:
coffees: list[tuple[str, str, tuple[str, ...]]] = [
    # (effect, coffee, variants)
    ("night_vision", "Auromar Geisha", ("long",)),
    ("leaping", "Legender Especial", ("long", "strong")),
    ("fire_resistance", "La Hermosa", ("long",)),
    ("swiftness", "Yirgacheffe", ("long", "strong")),
    ("water_breathing", "Kona", ("long",)),
    ("healing", "Ndumberi Peaberry", ("strong",)),
    ("regeneration", "Cimitrio Luna", ("long", "strong")),
    ("strength", "Minga Cabrera", ("long", "strong")),
    ("slow_falling", "Xejuyu", ("long",)),
]

template = """
execute if block ~ ~ ~
minecraft:brewing_stand{Items:[{components:{"minecraft:potion_contents":{potion:"minecraft:MODIFIER_EFFECT"}}}]}
run data modify block ~ ~ ~ Items[{components:{"minecraft:potion_contents":{potion:"minecraft:MODIFIER_EFFECT"}}}]
merge value {components:{"minecraft:custom_model_data":{"floats":[CMD]}, "minecraft:custom_name":'"NAME (SIZE)"'}}
""".replace(
    "\n", " "
).strip()

In [3]:
for effect, coffee, variants in coffees:
    print()
    base = template.replace("EFFECT", effect).replace("NAME", coffee)
    print(base.replace("MODIFIER_", "").replace("CMD", "1").replace("SIZE", "12oz"))
    if "long" in variants:
        print(
            base.replace("MODIFIER_", "long_")
            .replace("CMD", "2")
            .replace("SIZE", "16oz")
        )
    if "strong" in variants:
        print(
            base.replace("MODIFIER_", "strong_")
            .replace("CMD", "3")
            .replace("SIZE", "Espresso")
        )


execute if block ~ ~ ~ minecraft:brewing_stand{Items:[{components:{"minecraft:potion_contents":{potion:"minecraft:night_vision"}}}]} run data modify block ~ ~ ~ Items[{components:{"minecraft:potion_contents":{potion:"minecraft:night_vision"}}}] merge value {components:{"minecraft:custom_model_data":{"floats":[1]}, "minecraft:custom_name":'"Auromar Geisha (12oz)"'}}
execute if block ~ ~ ~ minecraft:brewing_stand{Items:[{components:{"minecraft:potion_contents":{potion:"minecraft:long_night_vision"}}}]} run data modify block ~ ~ ~ Items[{components:{"minecraft:potion_contents":{potion:"minecraft:long_night_vision"}}}] merge value {components:{"minecraft:custom_model_data":{"floats":[2]}, "minecraft:custom_name":'"Auromar Geisha (16oz)"'}}

execute if block ~ ~ ~ minecraft:brewing_stand{Items:[{components:{"minecraft:potion_contents":{potion:"minecraft:leaping"}}}]} run data modify block ~ ~ ~ Items[{components:{"minecraft:potion_contents":{potion:"minecraft:leaping"}}}] merge value {comp

### 3D Printing

In [4]:
custom_items: list[tuple[str | tuple[str, ...], str, int]] = [
    # name, base item, custom model data no.
    ("Aquarium Core", "carved_pumpkin", 3),
    ("Fact Sphere", "carved_pumpkin", 6),
    ("Morgan Freeman Core", "carved_pumpkin", 4),
    ("Rick", "carved_pumpkin", 7),
    ("Space Core", "carved_pumpkin", 5),
    ("Wheatley", "carved_pumpkin", 2),
    ("HIM", "carved_pumpkin", 1),
    ("Infinity Armor Statue", "carved_pumpkin", 1001),
    ("Infinity Armor Helmet", "carved_pumpkin", 1002),
    ("Mini-Him", "brown_mushroom", 1),
    (("Moonescent Pearl", "moonescent_pearl", "pearl"), "ender_pearl", 1),
    (("5am Pearl", "moonescent_pearl", "pearl"), "ender_eye", 1),
    ("Oddish", "grass", 1),
    ("Oddish", "seagrass", 1),
    (("Ancestor", "oddish"), "beetroot", 1),
    ("Shadow Dweller Mask", "carved_pumpkin", 1000),
    ("Steam Deck", "poisonous_potato", 1),
    ("Ghast Legs", "ghast_tear", 1),
    ("Jameson", "carved_pumpkin", 2001),
    (("Double-Pointed Needle", "dpn"), "stone_sword", 1),
    (("Double-Pointed Needle", "dpn"), "iron_sword", 1),
]

In [5]:
template = r"""
execute if block ~ ~1 ~ minecraft:hopper{Items:[{Slot:0b, id:"minecraft:{base_item}"}, {Slot:1b, id:"minecraft:paper", components:{"minecraft:custom_name": '"{tokenized}.gcode"'}}]} run data modify block ~ ~1 ~ Items[{Slot:0b}] merge value {components:{"minecraft:custom_model_data":{"floats":[{custom_model_num}]}, "minecraft:custom_name":'"{name}"'}}
execute if block ~ ~1 ~ minecraft:hopper{Items:[{Slot:0b, id:"minecraft:{base_item}"}, {Slot:1b, id:"minecraft:paper", components:{"minecraft:custom_name": '"{tokenized}.gcode"'}}]} run item replace entity @n[type=minecraft:glow_item_frame] container.0 from block ~ ~1 ~ container.0
execute if block ~ ~1 ~ minecraft:hopper{Items:[{Slot:0b, id:"minecraft:{base_item}"}, {Slot:1b, id:"minecraft:paper", components:{"minecraft:custom_name": '"{tokenized}.gcode"'}}]} run item replace block ~ ~1 ~ container.0 with minecraft:air
""".strip()

In [6]:
def tokenize(display_name: str) -> str:
    """Convert a display name into a suitable filename

    Parameters
    ----------
    display_name : str
        The original string

    Returns
    -------
    str
        The tokenized version of the name (all lowercase, spaces replaced with underscores)

    Examples
    --------
    >>> tokenize("Steam Deck")
    steam_deck
    """
    return display_name.lower().replace(" ", "_")

In [7]:
for names, base_item, custom_model_num in custom_items:
    if isinstance(names, str):
        name = names
        codes: tuple[str, ...] = (name,)
    else:
        name = names[0]
        codes = names[1:]
    for code in codes:
        print()
        print(
            template.replace("{name}", name)
            .replace("{base_item}", base_item)
            .replace("{custom_model_num}", str(custom_model_num))
            .replace("{tokenized}", tokenize(code))
        )


execute if block ~ ~1 ~ minecraft:hopper{Items:[{Slot:0b, id:"minecraft:carved_pumpkin"}, {Slot:1b, id:"minecraft:paper", components:{"minecraft:custom_name": '"aquarium_core.gcode"'}}]} run data modify block ~ ~1 ~ Items[{Slot:0b}] merge value {components:{"minecraft:custom_model_data":{"floats":[3]}, "minecraft:custom_name":'"Aquarium Core"'}}
execute if block ~ ~1 ~ minecraft:hopper{Items:[{Slot:0b, id:"minecraft:carved_pumpkin"}, {Slot:1b, id:"minecraft:paper", components:{"minecraft:custom_name": '"aquarium_core.gcode"'}}]} run item replace entity @n[type=minecraft:glow_item_frame] container.0 from block ~ ~1 ~ container.0
execute if block ~ ~1 ~ minecraft:hopper{Items:[{Slot:0b, id:"minecraft:carved_pumpkin"}, {Slot:1b, id:"minecraft:paper", components:{"minecraft:custom_name": '"aquarium_core.gcode"'}}]} run item replace block ~ ~1 ~ container.0 with minecraft:air

execute if block ~ ~1 ~ minecraft:hopper{Items:[{Slot:0b, id:"minecraft:carved_pumpkin"}, {Slot:1b, id:"minecraft:

#### Totems

In [8]:
with open("../_static/totem_of_undying.json") as model_file:
    custom_totems = json.load(model_file)["overrides"]

print(f"Model data specifies {len(custom_totems)} overrides")

Model data specifies 28 overrides


In [9]:
totem_list: list[tuple[int, str]] = []
for entry in custom_totems:
    model_num = int(entry["predicate"]["custom_model_data"])
    variant_name = entry["model"][len("item/totem_of_undying/") : -len("_totem")]
    if model_num == 914:
        continue
    print(f" - {model_num}: {variant_name}")
    totem_list.append((model_num, variant_name))

 - 901: docm77
 - 902: grian
 - 903: mumbo
 - 904: etho
 - 905: bdubs
 - 906: xisuma
 - 907: scar
 - 908: beef
 - 909: cleo
 - 910: cubfan
 - 911: hypno
 - 912: ijevin
 - 913: impulse
 - 915: joehills
 - 916: keralis
 - 917: tango
 - 918: tfc
 - 919: welsknight
 - 920: xb
 - 921: zedaph
 - 922: false
 - 923: pearl
 - 924: stress
 - 925: gem
 - 926: renking
 - 927: grumbot
 - 928: impulsedwarf


In [10]:
NAME_LOOKUP: dict[str, str] = {
    # I'm going by the stylizations of their names from hermitcraft.com
    # and omitting any cases where the default handling (title case) is correct
    "docm77": "the GOAT",
    "xisuma": "X",
    "bdubs": "Moss",
    "cubfan": "Cub",
    "ijevin": "iJevin",
    "joehills": "Joe",
    "renking": "the King",
    "tfc": "TFC",
    "welsknight": "Wels",
    "xb": "xB",  # tempted to use "princess"
    "zedaph": "Zed",
    "cleo": "the Zombie",
    "impulsedwarf": "Imli",  # creative license
}


def get_display_name(code: str) -> str:
    """Get the display name for a given custom totem

    Parameters
    ----------
    code : str
        The shortened name used to specify the model
        file that'll also be used for the "gcode"

    Returns
    -------
    str
        A suitable display name
    """
    return f"Totem of {NAME_LOOKUP.get(code, code.title())}"

In [11]:
for model_num, code in totem_list:
    print(f" - {model_num} : {get_display_name(code)}")

 - 901 : Totem of the GOAT
 - 902 : Totem of Grian
 - 903 : Totem of Mumbo
 - 904 : Totem of Etho
 - 905 : Totem of Moss
 - 906 : Totem of X
 - 907 : Totem of Scar
 - 908 : Totem of Beef
 - 909 : Totem of the Zombie
 - 910 : Totem of Cub
 - 911 : Totem of Hypno
 - 912 : Totem of iJevin
 - 913 : Totem of Impulse
 - 915 : Totem of Joe
 - 916 : Totem of Keralis
 - 917 : Totem of Tango
 - 918 : Totem of TFC
 - 919 : Totem of Wels
 - 920 : Totem of xB
 - 921 : Totem of Zed
 - 922 : Totem of False
 - 923 : Totem of Pearl
 - 924 : Totem of Stress
 - 925 : Totem of Gem
 - 926 : Totem of the King
 - 927 : Totem of Grumbot
 - 928 : Totem of Imli


In [12]:
template = r"""
execute if block ~ ~1 ~ minecraft:hopper{Items:[{Slot:1b, id:"minecraft:paper", components:{"minecraft:custom_name": '"{tokenized}.gcode"'}}]} run data modify block ~ ~1 ~ Items[{Slot:0b}] merge value {components:{"minecraft:custom_model_data":{"floats":[{custom_model_num}]}, "minecraft:custom_name":'"{name}"'}}
execute if block ~ ~1 ~ minecraft:hopper{Items:[{Slot:1b, id:"minecraft:paper", components:{"minecraft:custom_name": '"{tokenized}.gcode"'}}]} run item replace entity @n[type=minecraft:glow_item_frame] container.0 from block ~ ~1 ~ container.0
execute if block ~ ~1 ~ minecraft:hopper{Items:[{Slot:1b, id:"minecraft:paper", components:{"minecraft:custom_name": '"{tokenized}.gcode"'}}]} run item replace block ~ ~1 ~ container.0 with minecraft:air
""".strip()

for custom_model_num, code in totem_list:
    print()
    print(
        template.replace("{name}", get_display_name(code))
        .replace("{custom_model_num}", str(custom_model_num))
        .replace("{tokenized}", code)
    )


execute if block ~ ~1 ~ minecraft:hopper{Items:[{Slot:1b, id:"minecraft:paper", components:{"minecraft:custom_name": '"docm77.gcode"'}}]} run data modify block ~ ~1 ~ Items[{Slot:0b}] merge value {components:{"minecraft:custom_model_data":{"floats":[901]}, "minecraft:custom_name":'"Totem of the GOAT"'}}
execute if block ~ ~1 ~ minecraft:hopper{Items:[{Slot:1b, id:"minecraft:paper", components:{"minecraft:custom_name": '"docm77.gcode"'}}]} run item replace entity @n[type=minecraft:glow_item_frame] container.0 from block ~ ~1 ~ container.0
execute if block ~ ~1 ~ minecraft:hopper{Items:[{Slot:1b, id:"minecraft:paper", components:{"minecraft:custom_name": '"docm77.gcode"'}}]} run item replace block ~ ~1 ~ container.0 with minecraft:air

execute if block ~ ~1 ~ minecraft:hopper{Items:[{Slot:1b, id:"minecraft:paper", components:{"minecraft:custom_name": '"grian.gcode"'}}]} run data modify block ~ ~1 ~ Items[{Slot:0b}] merge value {components:{"minecraft:custom_model_data":{"floats":[902]},

#### Eggs

In [13]:
with open("../_static/eggs.json") as model_file:
    custom_eggs = json.load(model_file)["overrides"]

print(f"Model data specifies {len(custom_eggs)} overrides")

Model data specifies 19 overrides


In [14]:
egg_list: dict[int, str] = {}
for entry in custom_eggs:
    model_num = int(entry["predicate"]["custom_model_data"])
    filename = entry["model"][len("item/egg/") :]
    if model_num == 2230:
        continue
    print(f" - {model_num}: {filename}")
    egg_list[model_num] = filename

 - 2220: gold
 - 2221: bdubs
 - 2222: beef
 - 2223: cleo
 - 2224: cub
 - 2225: docm
 - 2226: false
 - 2227: grian
 - 2228: hypno
 - 2229: impulse
 - 2231: jevin
 - 2232: joe
 - 2233: pearl
 - 2234: scar
 - 2235: stress
 - 2236: wels
 - 2237: xb
 - 2238: xisuma
 - 2239: zed


In [15]:
egg_list[2224] = "cubfan"
egg_list[2225] = "docm77"
egg_list[2231] = "ijevin"
egg_list[2232] = "joehills"
egg_list[2236] = "welsknight"
egg_list[2239] = "zedaph"

In [16]:
name_lookup: dict[str, str] = {
    "gold": "Golden Egg",
    "bdubs": "Mossy Egg",
    "beef": "Beefy Egg",
    "cleo": "Zombie Clegg",
    "cubfan": "Cub Egg",
    "docm77": "GOATed Egg",
    "false": "False Egg",
    "grian": "Gregg",
    "hypno": "Hypno Egg",
    "impulse": "Impulsive Egg",
    "ijevin": "Jevin Egg",
    "joehills": "Egg of Joe",
    "pearl": "Pearled Egg",
    "scar": "Scarred Egg",
    "stress": "Stress Egg",
    "welsknight": "Wels Egg",
    "xb": "Princess Egg",
    "xisuma": "X Egg",
    "zedaph": "Zed Egg",
}

In [17]:
for model_num, code in egg_list.items():
    print(f" - {model_num} : {name_lookup[code]}")

 - 2220 : Golden Egg
 - 2221 : Mossy Egg
 - 2222 : Beefy Egg
 - 2223 : Zombie Clegg
 - 2224 : Cub Egg
 - 2225 : GOATed Egg
 - 2226 : False Egg
 - 2227 : Gregg
 - 2228 : Hypno Egg
 - 2229 : Impulsive Egg
 - 2231 : Jevin Egg
 - 2232 : Egg of Joe
 - 2233 : Pearled Egg
 - 2234 : Scarred Egg
 - 2235 : Stress Egg
 - 2236 : Wels Egg
 - 2237 : Princess Egg
 - 2238 : X Egg
 - 2239 : Zed Egg


In [18]:
for custom_model_num, code in egg_list.items():
    print()
    print(
        template.replace("{name}", name_lookup[code])
        .replace("{custom_model_num}", str(custom_model_num))
        .replace("{tokenized}", code)
    )


execute if block ~ ~1 ~ minecraft:hopper{Items:[{Slot:1b, id:"minecraft:paper", components:{"minecraft:custom_name": '"gold.gcode"'}}]} run data modify block ~ ~1 ~ Items[{Slot:0b}] merge value {components:{"minecraft:custom_model_data":{"floats":[2220]}, "minecraft:custom_name":'"Golden Egg"'}}
execute if block ~ ~1 ~ minecraft:hopper{Items:[{Slot:1b, id:"minecraft:paper", components:{"minecraft:custom_name": '"gold.gcode"'}}]} run item replace entity @n[type=minecraft:glow_item_frame] container.0 from block ~ ~1 ~ container.0
execute if block ~ ~1 ~ minecraft:hopper{Items:[{Slot:1b, id:"minecraft:paper", components:{"minecraft:custom_name": '"gold.gcode"'}}]} run item replace block ~ ~1 ~ container.0 with minecraft:air

execute if block ~ ~1 ~ minecraft:hopper{Items:[{Slot:1b, id:"minecraft:paper", components:{"minecraft:custom_name": '"bdubs.gcode"'}}]} run data modify block ~ ~1 ~ Items[{Slot:0b}] merge value {components:{"minecraft:custom_model_data":{"floats":[2221]}, "minecraft

## Resource Pack

So this one is fun. The old way model overrides work is that you place a file in `assets/minecraft/models/items` that looks like this:

```json
{
    "parent": "minecraft:item/generated",
    "textures": {
        "layer0": "minecraft:item/ghast_tear"
    },
    "overrides": [
        {
            "predicate": {
                "custom_model_data": 1
            },
            "model": "item/custom/ghast_butt"
        }
    ]
}
```

The _new_ way to do it is to create a file in `assets/minecraft/items` with _this_ syntax:

```json
{
  "model": {
    "type": "range_dispatch",
    "property": "custom_model_data",
    "fallback": {
      "type": "model",
      "model": "item/ghast_tear"
    },
    "entries": [
      {
        "threshold": 1,
        "model": {
          "type": "model",
          "model": "item/custom/ghast_butt"
        }
      }
    ]
  }
}
```

Luckily it's all programmatic I guess.

In [19]:
def convert_item_model(model_name: str, old_model: dict) -> dict:
    """Take a model item spec from the old (pack_format34-)
    sytax and generate a new (pack_format 46) syntax spec

    Parameters
    ----------
    model_name : str
        The name of the base model (probably the same as
        the name of the file)
    old_model : dict
        A model dict as would have worked in pack_format 34

    Returns
    -------
    dict
        A model dict that will work in pack_format 46
    """
    entries: list[dict] = []
    for entry in old_model["overrides"]:
        entries.append(
            {
                "threshold": entry["predicate"]["custom_model_data"],
                "model": {"type": "model", "model": entry["model"]},
            }
        )

    return {
        "model": {
            "type": "range_dispatch",
            "property": "custom_model_data",
            "fallback": {"type": "model", "model": f"item/{model_name}"},
            "entries": entries,
        }
    }

Try it out

In [20]:
display(
    Markdown(
        "```json\n"
        + json.dumps(
            convert_item_model(
                "egg",
                json.loads((Path("..") / "_static" / "eggs.json").read_text()),
            ),
            indent=2,
        )
        + "\n```"
    )
)

```json
{
  "model": {
    "type": "range_dispatch",
    "property": "custom_model_data",
    "fallback": {
      "type": "model",
      "model": "item/egg"
    },
    "entries": [
      {
        "threshold": 2220,
        "model": {
          "type": "model",
          "model": "item/egg/gold"
        }
      },
      {
        "threshold": 2221,
        "model": {
          "type": "model",
          "model": "item/egg/bdubs"
        }
      },
      {
        "threshold": 2222,
        "model": {
          "type": "model",
          "model": "item/egg/beef"
        }
      },
      {
        "threshold": 2223,
        "model": {
          "type": "model",
          "model": "item/egg/cleo"
        }
      },
      {
        "threshold": 2224,
        "model": {
          "type": "model",
          "model": "item/egg/cub"
        }
      },
      {
        "threshold": 2225,
        "model": {
          "type": "model",
          "model": "item/egg/docm"
        }
      },
      {
        "threshold": 2226,
        "model": {
          "type": "model",
          "model": "item/egg/false"
        }
      },
      {
        "threshold": 2227,
        "model": {
          "type": "model",
          "model": "item/egg/grian"
        }
      },
      {
        "threshold": 2228,
        "model": {
          "type": "model",
          "model": "item/egg/hypno"
        }
      },
      {
        "threshold": 2229,
        "model": {
          "type": "model",
          "model": "item/egg/impulse"
        }
      },
      {
        "threshold": 2231,
        "model": {
          "type": "model",
          "model": "item/egg/jevin"
        }
      },
      {
        "threshold": 2232,
        "model": {
          "type": "model",
          "model": "item/egg/joe"
        }
      },
      {
        "threshold": 2233,
        "model": {
          "type": "model",
          "model": "item/egg/pearl"
        }
      },
      {
        "threshold": 2234,
        "model": {
          "type": "model",
          "model": "item/egg/scar"
        }
      },
      {
        "threshold": 2235,
        "model": {
          "type": "model",
          "model": "item/egg/stress"
        }
      },
      {
        "threshold": 2236,
        "model": {
          "type": "model",
          "model": "item/egg/wels"
        }
      },
      {
        "threshold": 2237,
        "model": {
          "type": "model",
          "model": "item/egg/xb"
        }
      },
      {
        "threshold": 2238,
        "model": {
          "type": "model",
          "model": "item/egg/xisuma"
        }
      },
      {
        "threshold": 2239,
        "model": {
          "type": "model",
          "model": "item/egg/zed"
        }
      }
    ]
  }
}
```

Looking good. Now stick with me while I run this in bulk on the actual resource pack.

In [21]:
models_item_folder = (
    Path("..")
    / ".."
    / "chappeau"
    / "Chappeau Resourcepack"
    / "assets"
    / "minecraft"
    / "models"
    / "item"
)
assert models_item_folder.exists()

items_folder = models_item_folder / ".." / ".." / "items"
items_folder.mkdir(exist_ok=True)

In [22]:
for item_file in models_item_folder.glob("*.json"):
    print(f"Converting {item_file.stem}")
    (items_folder / item_file.name).write_text(
        json.dumps(
            convert_item_model(
                item_file.stem,
                json.loads(item_file.read_text()),
            ),
            indent=2,
        )
    )

Converting grass
Converting poisonous_potato
Converting stone_sword
Converting carved_pumpkin
Converting beetroot
Converting short_grass
Converting totem_of_undying
Converting ghast_tear
Converting brown_mushroom
Converting egg
Converting potion
Converting iron_sword
Converting seagrass
Converting ender_pearl
Converting ender_eye


And hopefully that's it!